In [ ]:
# require at least Python 3.5 for async/await to work
import sys
ver=sys.version_info
assert (ver[0]>=3 and ver[1]>=5)
sys.path.append('../aiostreams')
sys.path.append('../../msq-domain')

import aiostreams.operators.operators as op
from aiostreams.base import to_async_iterable

In [2]:
import copy

class Hedger :
    '''
    A basic hedge trade generator
    After each trade, returns a list containing the trade plus the hedge trades, 
    if any; after each quote returns an empty list
    so should be followed by a flattener.
    The hedge trades returned should have hedge.parent_id = trade.trade_id
    and hedge.paper_trade = True.
    It assumes all messages arrrive in strict timestamp order
    '''
    def __init__(self, hedge_calculator):
        self.hedge_calculator = hedge_calculator
        self.last_quotes = {}

    def __call__(self, msg):
        if isinstance(msg, Quote):
            # store the latest quote for each instrument; 
            # make a copy just in case the msg object gets modififed downstream
            self.last_quotes[msg.sym] = copy.deepcopy(msg)
            
            return []

        elif isinstance(msg, Trade):
            hedges = self.hedge_calculator(msg, self.last_quotes)
            
            return hedges + [msg]
        
        else:
            raise ValueError('Message must be a subclass of either Quote or Trade!')


In [3]:
#example use
topic1 = 'bund_trades'
topic2 = 'bund_quotes'
topic3 = 'hedge_quotes'   
    
src1= AsyncKafkaSource(topic1, kafka_host)

src=merge_sorted([src1, src2, src3])

def my_hedge_calculator(msg, lastquotes):
    return []

a = []
src | op.map(Hedger(my_hedge_calculator)) | op.flatten > a
print a

SyntaxError: invalid syntax (<ipython-input-3-58c6880be26f>, line 10)